NetworkX-Temporal
---

**[PyPI package](https://pypi.org/p/networkx-temporal/)** | **[Documentation](https://networkx-temporal.readthedocs.io/en/stable)** | **[GitHub repository](https://github.com/nelsonaloysio/networkx-temporal)**


In [ ]:
# !pip install 'networkx-temporal[ipynb]'   # Installs additional libraries used in this notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import networkx_temporal as tx

_____

# Basic operations

The examples below cover the package's basic functionalities, including
how to build a temporal graph, slice it into snapshots, save and load
graph objects to disk, and other inherited methods.


## Build temporal graph

This package implements new [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.graph.TemporalGraph)
classes, which extend [NetworkX
graphs](https://networkx.org/documentation/stable/reference/classes/index.html)
to handle temporal (dynamic) data. Let's start by creating a simple
directed graph using `time` as attribute key:

In [ ]:
TG = tx.TemporalMultiDiGraph()
# TG = tx.temporal_graph(directed=True, multigraph=True)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

print(TG)

Note that the resulting graph object reports a single time step `t=1`,
as it has not yet been [sliced](#slice-temporal-graph).

> **Attention:**
> To allow multiple interactions between the same nodes over time, a
> [`TemporalMultiGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.graph.TemporalMultiGraph)
> or [`TemporalMultiDiGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.graph.TemporalMultiDiGraph)
> object is required. Otherwise, only a single edge is allowed among pairs.

## Import static graphs

Static graph objects with temporal information may also be imported as
temporal graphs.

In [ ]:
G = nx.DiGraph()

G.add_nodes_from([
    ("a", {"time": 0}),
    ("b", {"time": 0}),
    ("c", {"time": 1}),
    ("d", {"time": 2}),
    ("e", {"time": 3}),
    ("f", {"time": 3}),
])

G.add_edges_from([
    ("a", "b", {"time": 0}),
    ("c", "b", {"time": 1}),
    ("d", "c", {"time": 2}),
    ("d", "e", {"time": 2}),
    ("a", "c", {"time": 2}),
    ("f", "e", {"time": 3}),
    ("f", "a", {"time": 3}),
    ("f", "b", {"time": 3}),
])

print(G)

We may convert the graph above to a
[`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.TemporalGraph) object using the
[`from_static`](https://networkx-temporal.readthedocs.io/en/stable/api/transform.html#networkx_temporal.transform.from_static) function. As expected, the
resulting object has the same total number of nodes and edges as the
original graph:

In [ ]:
TG = tx.from_static(G)

# assert G.order() == TG.order(copies=False)
# assert G.size() == TG.size(copies=True)

print(TG)

The [`draw`](https://networkx-temporal.readthedocs.io/en/stable/api/drawing.html#networkx_temporal.drawing.draw)
function allows to visualize the
edge-level temporal information in a single plot:

In [ ]:
tx.draw(TG, layout="kamada_kawai", edge_labels="time", suptitle="Temporal Graph")

However, note that in the example above, both the nodes and edges
contain a `time` attribute. Let's see next how this affects the
resulting temporal graph when slicing the graph data into snapshots.

> **See also:**
> The [`from_snapshots`](https://networkx-temporal.readthedocs.io/en/stable/api/transform.html#networkx_temporal.transform.from_snapshots)
> function to import a
> list of static graphs as temporal graph snapshots.

## Slice temporal graph

Let's use the [`slice`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.slice) method to
split the temporal graph we created into a number of snapshots:

In [ ]:
TG = TG.slice(attr="time")
print(TG)

Inspecting the resulting object's properties using some
familiar methods:

In [ ]:
print(f"t = {len(TG)} snapshots",   # TG.number_of_graphs()
      f"V = {TG.order()} nodes",    # TG.number_of_nodes()
      f"E = {TG.size()} edges",     # TG.number_of_edges()
      sep="\n")

We may also visualize the resulting snapshots using the
[`draw`](https://networkx-temporal.readthedocs.io/en/stable/api/drawing.html#networkx_temporal.drawing.draw) function:

In [ ]:
tx.draw(TG, layout="kamada_kawai", figsize=(8, 2))

Note that [`slice`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.slice) by default
returns a snapshot for each unique attribute value in the graph.

> **Hint:**
> By default, [`slice`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.slice) returns the
> interval of the resulting snapshots as their
> [`names`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.TemporalGraph.names) property. Passing
> `title=True` to [`draw`](https://networkx-temporal.readthedocs.io/en/stable/api/drawing.html#networkx_temporal.draw) will use them instead
> of indices as subplot titles, as seen below.

### Number of snapshots

A new object can be created with a specific number of snapshots by
setting the `bins` parameter:

In [ ]:
TG = TG.slice(attr="time", bins=2)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), title=True)

> **Note:**
> In case [`slice`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.slice) is not able to split the graph into
> the number specified by `bins` (e.g., due to insufficient
> data), the maximum possible number of snapshots is returned instead.

### Quantile-based cut

Setting `qcut=True` slices a graph into quantiles, creating snapshots
with balanced order and/or size (nodes/edges). This is useful when
interactions are not evenly distributed across time. For example:

In [ ]:
TG = TG.slice(attr="time", bins=2, qcut=True)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), title=True)

The resulting snapshots have uneven time intervals: $t=(0,2]$ and
$t=(2,3]$, respectively. Objects are sorted by their `time` attribute
values and then split into two groups with approximately the same order
(nodes) or size (edges), depending on the level of the attribute passed
to the function.

> **See also:**
> The [pandas.qcut
> documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)
> for more information on quantile-based discretization.

### Rank-based cut

Setting `rank_first=True` slices a graph considering the order of
appearance of edges (default), nodes, or attributes, forcing each
snapshot to have approximately the same number of objects:

In [ ]:
TG = TG.slice(bins=2, rank_first=True)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), title=True)

As `attr` was not set, the graph was split considering the order in
which edges were added to the graph. Notice how each snapshot title now
refer to edge intervals: $e_0$ to $e_3$ $(0, 4]$ and $e_4$ to $e_7$
$(4, 8]$. This is useful to obtain an arbitrary number of subgraphs,
independent of their temporal dynamics.

> **See also:**
> The [pandas.rank
> documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.rank.html)
> for more information on ranking data.


### Edge-level time attribute

Converting a static graph considering edge-level temporal data into a
temporal graph object:

In [ ]:
TG = TG.slice(attr="time")
tx.draw(TG, layout="kamada_kawai", figsize=(8, 2))

The resulting temporal graph has the same number of edges as the
original graph, but a higher number of nodes. This is expected, as the
same nodes appear in more than one snapshot.

> **Note:**
> By default, [`slice`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.slice)
> considers `attr` as an edge-level attribute, which is usually the case for temporal data.
> This behavior can be changed by setting `level='node'`, as seen below.

### Node-level time attribute

Converting a static graph considering node-level temporal data to a
temporal graph object:

In [ ]:
TG = TG.slice(attr="time", level="node")
tx.draw(TG, layout="kamada_kawai", figsize=(8, 2))

Note that now, even though the edge $(a, c)$ contains the attribute
`time=2`, considering node-level attributes resulted in it being
placed at the snapshot $t=0$ instead, as node $a$ is set to `time=0`:

In [ ]:
G.nodes(data="time")["a"]

> **Note:**
> When `level='node'`, the source node's ``attr`` value is used by
> default to determine the edge's interaction time. This behavior can be
> changed by setting `level='target'` instead.


## Save and load data

Temporal graphs may be read from or written to a file using
[`read_graph`](https://networkx-temporal.readthedocs.io/en/stable/api/io.html#networkx_temporal.read_graph) and
[`write_graph`](https://networkx-temporal.readthedocs.io/en/stable/api/io.html#networkx_temporal.write_graph):

In [ ]:
tx.write_graph(TG, "temporal-graph.graphml.zip")
TG = tx.read_graph("temporal-graph.graphml.zip")

Supported formats are the same as those in NetworkX and depend on the version installed.

> **See also:**
> The [read and write
> documentation](https://networkx.org/documentation/stable/reference/readwrite/index.html)
> from NetworkX for a list of supported graph formats.

## Inherited methods

Any methods available from a [NetworkX
graph](https://networkx.org/documentation/stable/reference/classes/graph.html#networkx.Graph)
can be called directly from a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) object.
For example, the familiar methods below transform edges in the graph
into directed or undirected:

In [ ]:
TG.to_undirected()

In [ ]:
TG.to_directed()

Note that both methods return new objects when called, so the original
graph remains unchanged. Additional utility functions for temporal
graphs are available in the [`utils`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html) module.

> **See also:**
> -   The [Appendix → Index](genindex.html) page for a list of the implemented classes, methods, and functions.
> -   The [NetworkX
>     documentation](https://networkx.org/documentation/stable/reference/classes/graph.html#methods)
>     for a list of methods inherited by a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph)
>     object.

___

# Algorithms and metrics

Algorithms implemented by NetworkX can be called on graph snapshots, while
[NetworkX graph](https://networkx.org/documentation/stable/reference/classes/graph.html#networkx.Graph)
methods are inherited by
[`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph)
objects.
This section highlights a few common examples.

## Order and size

The methods [`order`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.algorithms.graph.order) and
[`size`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.algorithms.graph.size)
return the number of
nodes and edges in each graph snapshot, respectively, while an
additional argument `copies` allows specifying whether to count
duplicates:

In [ ]:
import networkx as nx
import networkx_temporal as tx

TG = tx.TemporalMultiDiGraph()
# TG = tx.temporal_graph(directed=True, multigraph=True)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=0)
TG.add_edge("c", "b", time=1)   # <-- parallel edge
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

TG = TG.slice(attr="time")
print(TG)

Note that when printing a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.TemporalGraph)
instance, the order of the graph $|\mathcal{V}|$ corresponds to the
number of unique nodes and its size to the number of edge interactions
$|\mathcal{E}|$ (with parallel edges):

In [ ]:
print("Order:", TG.order())
print("Order (unique nodes):", TG.order(copies=False))
print("Order (including copies):", TG.order(copies=True))

In [ ]:
print("Size:", TG.size())
print("Size (unique edges):", TG.size(copies=False))
print("Size (including copies):", TG.size(copies=True))

Visualizing the graph with [`draw`](https://networkx-temporal.readthedocs.io/en/stable/api/drawing.html#networkx_temporal.drawing.draw),
however, shows all
nodes and edges, including their copies:

In [ ]:
tx.draw(TG, layout="kamada_kawai", figsize=(8,2))

Note that when printing a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) instance, the order of
the graph $|\mathcal{V}|$ corresponds to the number of unique nodes and its size to the
number of edge interactions $|\mathcal{E}|$ (including copies).

> **See also:**
The alias methods:
[`temporal_order`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.temporal_order),
[`temporal_size`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.temporal_size),
[`total_order`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.total_order),
and [`total_size`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.total_size).

## Graph centralization

[Centralization](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.algorithms.graph.centralization) [1]
is a graph-level metric that compares the sum of all
node centralities against the maximum possible score for a graph with
the same properties, e.g., order, size, and directedness.

### Degree centralization

The [`degree_centralization`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.algorithms.graph.degree_centralization)
function returns the score considering node degrees per snapshot:

In [ ]:
tx.degree_centralization(TG)
# tx.in_degree_centralization(TG)
# tx.out_degree_centralization(TG)

> **Note:**
> In directed graphs, the in-degree and out-degree centralization scores
> may differ:

In [ ]:
tx.in_degree_centralization(TG) == tx.out_degree_centralization(TG)

In [ ]:
for t, G in enumerate(TG):
    indc = tx.in_degree_centralization(G)
    outdc = tx.out_degree_centralization(G)
    print(f"t={t}: in={indc:.2f}, out={outdc:.2f}")

For example, the theoretical graph corresponding to the maximum degree
centralization is a [star-like
structure](https://networkx.org/documentation/stable/reference/generated/networkx.generators.classic.star_graph.html),
where one central node is connected to all other nodes in the graph,
which are not connected among themselves. Such a graph corresponds to a
degree centralization score of $1.0$:

In [ ]:
G = nx.star_graph(10)
tx.draw(G, layout="fruchterman_reingold")

Note that while edge directedness is considered, self-loops and isolates
are ignored by default. This behavior may be changed by passing
`isolates=True` or `self_loops=True` arguments, respectively:

In [ ]:
H = G.copy()

H.add_node(-1)    # Disconnected node.
H.add_edge(0, 0)  # Edge self-loop.

print(f"Default: {tx.degree_centralization(H)}\n"
      f"With isolates: {tx.degree_centralization(H, isolates=True):.3f}\n"
      f"With self-loops: {tx.degree_centralization(H, self_loops=True):.3f}\n"
      f"With both: {tx.degree_centralization(H, isolates=True, self_loops=True):.3f}")

### More centralization metrics

In [ ]:
centrality = G.degree()
scalar = sum(G.order() - 2 for n in range(G.order()-1))  # |V|-1 for DiGraph

centralization = tx.centralization(centrality=centrality, scalar=scalar)
print(f"Degree centralization: {centralization:.2f}")

The function may be used to calculate the score for other centrality
measures, e.g., closeness and betweenness, where the most centralized
structure is a star-like graph, or eigenvector centrality, where the
most centralized structure is a graph with a single edge (and
potentially many isolates).

> **See also:**
> The [igraph documentation on the centralization of a
> graph](https://igraph.org/r/html/1.3.5/centralize.html) for additional
> implementations.

## Node centrality

The functions and algorithms implemented by NetworkX can be applied
directly on the temporal graph by iterating over snapshots. For instance,
to calculate the [Katz centrality](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.katz_centrality.html) for each snapshot:

In [ ]:
TG = tx.from_multigraph(TG)

for t, G in enumerate(TG):
    katz = nx.katz_centrality(G)
    katz = {node: round(value, 2) for node, value in katz.items()}
    print(f"t={t}: {katz}")

Note that we first converted the multigraph to a simple graph (without parallel edges) using the
`from_multigraph` function, as the algorithm implementation does not support multigraphs.

> **See also:**
> The [Algorithms section](https://networkx.org/documentation/stable/reference/algorithms/index.html)
> of the NetworkX documentation for a list of available functions.

### Node degree

In addition, any NetworkX [graph
methods](https://networkx.org/documentation/stable/reference/classes/graph.html#methods)
can be called directly from the temporal graph.
For example, the methods
[`degree`](https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.degree.html),
[`in_degree`](https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.in_degree.html),
and
[`out_degree`](https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.out_degree.html)
return the results per snapshot:

In [ ]:
TG.degree()
# TG.in_degree()
# TG.out_degree()

In [ ]:
TG.degree("a")

Note that, for $a$ in $t=1$ above, a `None` is returned as the node is
not present in that snapshot.

### Degree centrality

Alternatively, [`degree_centrality`](https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.degree.html)
returns the fraction of nodes connected to each of all nodes:

In [ ]:
tx.degree_centrality(TG)
# tx.in_degree_centrality(TG)
# tx.out_degree_centrality(TG)

In [ ]:
tx.degree_centrality(TG, "a")
# tx.in_degree_centrality(TG, "a")
# tx.out_degree_centrality(TG, "a")

Note that the degree centrality is calculated considering the unique
nodes on the whole graph.

### Total degree

Likewise, the [`degree`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.algorithms.node.degree)
function returns instead a dictionary with the sum of node degrees over time:

In [ ]:
tx.degree(TG)
# tx.in_degree(TG)
# tx.out_degree(TG)

In [ ]:
tx.degree(TG, "a")
# tx.in_degree(TG, "a")
# tx.out_degree(TG, "a")


> **See also:**
> The alias methods
> [`total_degree`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.total_degree),
> [`total_in_degree`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.total_in_degree),
> and [`total_out_degree`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.total_out_degree),
> which return a dictionary with the sum of node degrees over time,
> while maintaining its original ordering.

## Neighbors

Edge directedness is considered when obtaining the neighbors of a node,
either per snapshot or considering all snapshots, via the
[`neighbors`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.TemporalGraph.neighbors)
method and the
[`neighbors`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.neighbors) function, respectively.

### Per snapshot

The [`neighbors`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.TemporalGraph.neighbors)
method returns a generator over graph snapshots, respecting edge direction:

In [ ]:
TG = TG.slice(attr="time")
list(TG.neighbors("c"))

Converting the graph to undirected, we also obtain nodes that have node
$c$ as their neighbor:

In [ ]:
list(TG.to_undirected().neighbors("c"))

> **Hint:**
> The above is effectively the same as calling the
> [`all_neighbors`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.all_neighbors)
> method instead.

#### From all snapshots

[`neighbors`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.neighbors) function
returns a node neighborhoods considering all snapshots:

In [ ]:
list(tx.neighbors(TG, "c"))

Converting the graph to undirected, we also obtain temporal nodes that have it
as their neighbor:

In [ ]:
list(tx.neighbors(TG.to_undirected(), "c"))

Indexes allow to restrict the search to specific snapshots in time, e.g., from $t=0$ to $t=1$:

In [ ]:
list(tx.neighbors(TG[0:2], "c"))

> **Note:** Indexing follows Python conventions and is inclusive on the left and exclusive on the right, i.e., the above example returns the neighbors of node $c$ at time steps $t=0$ and $t=1$.

___

# Convert and transform

This package provides a set of functions to manipulate graph classes, formats,
and representations. In this context, [`convert`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#module-networkx_temporal.utils.convert) refers to different graph-based libraries,
e.g., [igraph](https://igraph.org), and [`transform`](https://networkx-temporal.readthedocs.io/en/stable/api/transform.html#module-networkx_temporal.transform)
refers to the underlying data structure used to store object relations, e.g.,
[event-based temporal graphs](#event-based-temporal-graph).

## Graph classes

Static or temporal multigraphs may be converted to graphs without parallel edges and vice-versa:

In [ ]:
import networkx_temporal as tx

TG = tx.TemporalMultiDiGraph()
# TG = tx.temporal_graph(directed=True, multigraph=True)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=0)
TG.add_edge("c", "b", time=1)   # <-- parallel edge
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

TG = TG.slice(attr="time")
print(TG)

In the example above, the temporal multigraph `TG` was first sliced into
$t=4$ snapshots. Let's now convert it to a simple graph, in which
parallel edges among the same node pair are not allowed:

In [ ]:
TG = tx.from_multigraph(TG)
print(TG)

The resulting graph has the same size of $|\mathcal{E}| = 9$ edges, as
the parallel edges among nodes $c$ and $b$ were in different snapshots,
$t=\{0,1\}$. Let's now
[`flatten`](https://networkx-temporal.readthedocs.io/en/stable/api/classes.html#networkx_temporal.classes.TemporalGraph.flatten) it and call
[`from_multigraph`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.from_multigraph)
again:

In [ ]:
TG = tx.to_multigraph(TG)    # Restore temporal graph back to a multigraph.
TG = TG.flatten()            # Obtain temporal graph with a single snapshot.
TG = tx.from_multigraph(TG)  # Combine parallel edges into single ones.

print(TG)

In this case, the resulting graph has $|\mathcal{E}| = 8$ edges, as the
parallel edges among nodes $c$ and $b$ were combined into a single one
with the attributes `time=1` and `weight=2`, referring to the last
snapshot time and their total number of interactions, respectively,
resulting in an irreversible operation:

In [ ]:
TG.edges(("c", "b"), data=True)

> **Attention:**
The conversion from a temporal multigraph to a simple graph is not
always reversible, as dynamic edge attributes, e.g., `time`, are
overwritten when converting multigraphs to graphs.

## Graph formats

The [`read_graph`](https://networkx-temporal.readthedocs.io/en/stable/api/readwrite.html#networkx_temporal.readwrite.read_graph)
and [`write_graph`](https://networkx-temporal.readthedocs.io/en/stable/api/readwrite.html#networkx_temporal.readwrite.write_graph)
functions offer a high-level interface to load and store graph data, supporting any format
implemented in the current installed version of NetworkX.

In [ ]:
tx.write_graph(TG, "temporal-graph.graphml.zip")
TG = tx.read_graph("temporal-graph.graphml.zip")

> **See also:**
> The [NetworkX documentation](https://networkx.org/documentation/stable/reference/readwrite/index.html)
> for an and updated list of supported reading and writing formats.

## Graph libraries

Support for the following external libraries are currently implemented in the package:

| Format | Parameter (Package) | Calls (Function) |
|---|:---:|:---:|
| [Deep Graph Library](https://www.dgl.ai) | `dgl` | [`to_dgl`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_dgl)
| [DyNetX](https://dynetx.readthedocs.io) | `dynetx` | [`to_dynetx`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_dynetx)
| [graph-tool](https://graph-tool.skewed.de) | `graph_tool` | [`to_graph_tool`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_graph_tool)
| [igraph](https://igraph.org/python) | `igraph` | [`to_igraph`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_igraph)
| [NetworKit](https://networkit.github.io) | `networkit` | [`to_networkit`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_networkit)
| [PyTorch Geometric](https://pytorch-geometric.readthedocs.io) | `torch_geometric` | [`to_torch_geometric`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_torch_geometric)
| [SNAP](https://snap.stanford.edu) | `snap` | [`to_snap`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_snap)
| [StellarGraph](https://stellargraph.readthedocs.io) | `stellargraph` | [`to_stellargraph`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_stellargraph)
| [Teneto](https://teneto.readthedocs.io) | `teneto` | [`to_teneto`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.utils.convert.to_teneto)

Graphs may be converted to a different library format with the high-level
[`convert`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.convert) function.

In [ ]:
TG = TG.slice(attr="time")
tx.convert(TG, "igraph")

By default, the amount of objects returned match the number of slices (snapshots). To return a single [static graph](#static-graph) instead:

In [ ]:
tx.convert(TG.to_static(), "igraph")

___

## Graph representations

Once instantiated, [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph)
objects may be transformed into different representations, depending on
the analysis or visualization requirements. Due to the nature of temporal
graphs, some representations may not preserve all the data, such as
dynamic node or edge attributes.

Observe that the total number of returned nodes $V$ and edges $E$
after transformation might differ from the number of temporal nodes $V_T$ and
edges $E_T$ depending on the data and method used:

| Representation | Order | Size  | Dynamic node attributes | Dynamic edge attributes |
| --- | :---: | :---: | :---: | :---: |
| [Static](#static-graph) | $V = V_T$ | $E = E_T$ | ❌ | ✅ |
| [Snapshots](#snapshot-based-temporal-graph)* | $V \ge V_T$ | $E = E_T$ | ✅ | ✅ |
| [Events](#event-based-temporal-graph) | $V = V_T$ | $E = E_T$ | ❌ | ❌ |
| [Unrolled](#unrolled-temporal-graph) | $V \ge V_T$ | $E \ge E_T$ | ✅ | ✅ |

(\*) Default underlying data structure for temporal graphs with multiple snapshots on
[`slice`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.slice).

### Static graph

A static graph `G` is a single graph object containing all the nodes and
edges found in the temporal graph. It is the simplest representation of
a network and is the most common type of graph.

> **Attention:**
> Dynamic node attributes are not preserved when transforming a temporal
> to a static graph.

> **See also:**:
> The [Basic operations → Import static
> graphs](https://networkx-temporal.readthedocs.io/en/stable/examples/basics.html#import-static-graphs) page for more static graph
> conversion examples.

#### `TG` → `G`

Transforming a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) into a static graph with the
[`to_static`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.to_static) method:

In [ ]:
G = TG.to_static()
print(G)

In [ ]:
tx.draw(G, layout="kamada_kawai", suptitle="Static Graph")

#### `G` → `TG`

Transforming a static graph into a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) with the
[`from_static`](https://networkx-temporal.readthedocs.io/en/stable/api/utils.html#networkx_temporal.from_static) function:

In [ ]:
TG = tx.from_static(G)
TG = TG.slice(attr="time")
print(TG)

### Snapshot-based temporal graph

A snapshot-based temporal graph `STG` is a sequence of graphs where each
element represents a snapshot of the original temporal graph. It is the
most common representation of temporal graphs.

> **Note:**
> Like the [`slice`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.slice) method,
> [`to_snapshots`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.to_snapshots) internally returns views
> of the original graph data, so no data is copied unless specified
> otherwise, i.e., by passing `as_view=False` to the function.

#### `TG` → `STG`

Transforming a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) into a snapshot-based temporal graph with
[`to_snapshots`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.to_snapshots):

In [ ]:
STG = TG.to_snapshots()
STG

#### `STG` → `TG`

Transforming a snapshot-based temporal graph into a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) with
[`from_snapshots`](https://networkx-temporal.readthedocs.io/en/stable/api/transform.html#networkx_temporal.from_snapshots):

In [ ]:
TG = tx.from_snapshots(STG)
print(TG)

### Event-based temporal graph

An event-based temporal graph `ETG` is a sequence of 3- or 4-tuple
edge-based events.

-   **3-tuples** ($u, v, t$), where elements are the source node, target
    node, and time attribute;
-   **4-tuples** ($u, v, t, \delta$), where $\delta$ is either an
    `int` for edge addition ($1$) or deletion ($-1$) events, or a `float`
    for the duration of the interaction (zero for a single snapshot).

Depending on the temporal graph data, one of these may allow a more
compact representation than the others. The default is to return a
3-tuple sequence (also known as a *stream graph*).

> **Important**:
> As events are edge-based, node isolates without self-loops are not preserved.

#### `TG` → `ETG`

Transforming a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) into an event-based temporal graph with
[`to_events`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.to_events):

In [ ]:
ETG = TG.to_events()  # eps=None
ETG

In [ ]:
ETG = TG.to_events(delta=int)
ETG

In [ ]:
ETG = TG.to_events(delta=float)
ETG

#### `ETG` → `TG`

Transforming an event-based temporal graph into a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) with
[`from_events`](https://networkx-temporal.readthedocs.io/en/stable/api/transform.html#networkx_temporal.from_events):

In [ ]:
TG = tx.from_events(ETG, directed=True, multigraph=None)
print(TG)

### Unrolled temporal graph

An unrolled temporal graph `UTG` is a single graph object that contains the original temporal data,
plus additional time-adjacent node copies (from each snapshot) and edge couplings connecting them.
Its usefulness is restricted to certain types of analysis and visualization, e.g., based on
temporal flows.

> **See also:** For an example with temporal node centrality metrics, see
> [Hyoungshick & Anderson, 2012](https://doi.org/10.1103/PhysRevE.85.026107) [2].

#### `TG` → `UTG`

Transforming a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) into an unrolled temporal graph with
[`to_unrolled`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph.to_unrolled),
and drawing the resulting graph with node copies (in black) and edge couplings (dotted):

In [ ]:
UTG = TG.to_unrolled()
print(UTG)

Let's draw the resulting graph to visualize the node copies (in black) and edge couplings (dotted):

In [ ]:
node_color = [
    "tab:red" if int(n.split("_")[1]) == TG.index_node(n.split("_")[0])[0] else "#333"
    for n in UTG.nodes()]

tx.draw_unrolled(UTG, node_color=node_color, connectionstyle="arc3,rad=0.25",
                 title="Unrolled Temporal Graph")

> **Hint:**
> By default, edges connect nodes in the same snapshot, e.g., from $u_t$ to $v_t$.
> To create edges that connect nodes across time, e.g., from $u_t$ to
> $v_{t+\delta}$, pass the ``delta`` parameter to the function
> with the desired edge-level attribute, e.g., ``delta='duration'``,
> or time difference, e.g., ``delta=1``.

Passing ``delta=1`` to the function creates edges connecting nodes in adjacent snapshots.
In the following plot, blue nodes are those newly created and not present in the previous example:

In [ ]:
UTG_delta = TG.to_unrolled(delta=1)

node_color = [
    "tab:red" if UTG.has_node(n) else "tab:blue"
    for n in UTG_delta.nodes()]

tx.draw_unrolled(UTG_delta, node_color=node_color,
                 title="Unrolled Temporal Graph ($\\delta=1$)")

> **Attention:**
> New nodes and edges are created depending on the ``delta`` value passed to the
> function, leading to graphs of different order and size. For instance, passing
> ``delta=1`` in the example above created additional edges among node $f_3$
> and the temporal node copies $a_4$, $b_4$, and $e_4$.

Lastly, the additional parameters ``edge_couplings`` and ``node_copies`` allow further control over
the creation of new temporal node copies and edge couplings connecting them, as shown below:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8, 2))

tx.draw_unrolled(TG.to_unrolled(delta=1, node_copies="fill"),
                 node_size=200, fig=fig, ax=0, title="node_copies='fill'")

tx.draw_unrolled(TG.to_unrolled(delta=1, node_copies="persist"),
                 node_size=200, fig=fig, ax=1, title="node_copies='persist'")

tx.draw_unrolled(TG.to_unrolled(delta=1, node_copies="all"),
                 node_size=200, fig=fig, ax=2, title="node_copies='all'")

#### `UTG` → `TG`

Transforming an unrolled temporal graph into a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/stable/api/graph.html#networkx_temporal.graph.TemporalGraph) with
[`from_unrolled`](https://networkx-temporal.readthedocs.io/en/stable/api/transform.html#networkx_temporal.from_unrolled):

In [ ]:
TG = tx.from_unrolled(UTG)
# TG = tx.from_unrolled(UTG_delta)
print(TG)

_____

# Community detection

Community detection is a fundamental task in network analysis. This
simple example demonstrates how a network's temporal dynamics can
overall benefit the detection of its mesoscale structures.

## Generate graph

As a toy example, let's use the simplest [Stochastic Block
Model](https://networkx.org/documentation/stable/reference/generated/networkx.generators.community.stochastic_block_model.html)
to generate 4 graph snapshots, in which each of the 5 clusters of 5
nodes each continuously mix over time (decreasing assortativity):

In [ ]:
snapshots = 4   # Temporal graphs to generate.
clusters = 5    # Number of clusters/communities.
order = 5       # Nodes in each cluster.
intra = .9      # High initial probability of within-community edges.
inter = .1      # Low initial probability of inter-community edges.
change = .125   # Change in within- and inter-community edges over time.

# Get probability matrix for each snapshot.
probs = [[[
    (intra if i == j else inter) + (t * change * (-1 if i == j else 1))
    for j in range(clusters)]
    for i in range(clusters)]
    for t in range(snapshots)]

# Create graphs from probabilities.
graphs = {}
for t in range(snapshots):
    graphs[t] = nx.stochastic_block_model(clusters*[order], probs[t], seed=10)
    graphs[t].name = t

# Create temporal graph from snapshots.
TG = tx.from_snapshots(graphs)

Let's plot the graphs, with node colors representing communities and
within-community edges:

In [ ]:
import matplotlib.pyplot as plt
c = plt.cm.tab10.colors

def get_edge_color(edges: list, node_color: dict = node_color):
    return [node_color[u]
            if node_color[u] == node_color[v]
            else (0, 0, 0, 0.25)
            for u, v in edges]

# Node positions.
pos = nx.circular_layout(TG.to_static())

# Node options for all graphs; colorize nodes by community.
node_color = [c[i // clusters] for i in range(TG.temporal_order())]

# Edge options per snapshot; colorize within-community edges.
temporal_edge_color = {
    t: get_edge_color(TG[t].edges(), node_color)
    for t in range(len(TG))}

# Plot snapshots with community ground truths.
tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    node_size=300,
    node_color=node_color,
    temporal_edge_color=temporal_edge_color,
    suptitle="Ground truths")

We see that all snapshots are generated with the same community structure, but varying degrees of
assortativity. Let's try to retrieve the ground truths using a simple community detection algorithm.

## Modularity optimization

The [leidenalg](https://leidenalg.readthedocs.io) [3] package implements
optimization algorithms for community detection that may be applied on
snapshot-based temporal graphs, allowing to better capture their
underlying structure.

> **Attention:**
> Optimization algorithms may help with descriptive or exploratory
> tasks and post-hoc network analysis, but lack statistical rigor for
> inferential purposes. See [Peixoto
> (2021)](https://skewed.de/tiago/posts/descriptive-inferential/) [4] for
> a discussion.


### On the static graph

Let's start by considering the network as a single static graph,
ignoring its temporal information.

We can observe that, depending on the initial node community assigments
(e.g., with `seed=0` below),
[modularity](https://leidenalg.readthedocs.io/en/stable/reference.html#modularityvertexpartition)
fails to retrieve the true communities (ground truths) in the network:

In [ ]:
import leidenalg as la

membership = la.find_partition(
    TG.to_static("igraph"),
    la.ModularityVertexPartition,
    n_iterations=-1,
    seed=0,
)

node_color = [c[m] for m in membership.membership]
edge_color = get_edge_color(TG.to_static().edges(), node_color)

tx.draw(
    TG.to_static(),
    pos=pos,
    figsize=(4, 4),
    node_size=300,
    node_color=node_color,
    edge_color=edge_color,
    connectionstyle="arc3,rad=0.1",
    suptitle="Modularity optimization on static graph")

Next, let's try considering the network's temporal information to see if
we can improve the results.

### On each snapshot

Running the same algorithm separately on each of the generated snapshots
retrieves the correct clusters only on the first graph ($t=0$). In
addition, community indices (represented by their colors) are not fixed
over snapshots, which makes it harder to track their mesoscale dynamics:

In [ ]:
temporal_node_color, temporal_edge_color = {}, {}

for t in range(len(TG)):
    membership = la.find_partition(
        TG[t:t+1].to_static("igraph"),
        la.ModularityVertexPartition,
        n_iterations=-1,
        seed=0)
    temporal_node_color[t] = [c[m] for m in membership.membership]
    temporal_edge_color[t] = get_edge_color(TG[t].edges(), temporal_node_color[t])

tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    node_size=300,
    temporal_node_color=temporal_node_color,
    temporal_edge_color=temporal_edge_color,
    connectionstyle="arc3,rad=0.1",
    suptitle="Modularity optimization on graph snapshots")

This is partly due to modularity optimization expecting an assortative
community structure, while the network grew more disassortative over
time. Not only the results of later snapshots are here suboptimal, but
the varying community indices increase the complexity of their temporal
analysis.

### On the temporal graph

Considering snapshots as layers (slices) of a multiplex graph, with
[interslice edges coupling temporal node copies](https://leidenalg.readthedocs.io/en/stable/multiplex.html#slices-to-layers),
is one way of employing modularity optimization on dynamic graphs, which may help to better capture
their mesoscale structures [5]. This example uses the same algorithm as before:

In [ ]:
temporal_membership, improvement = la.find_partition_temporal(
    TG.to_snapshots("igraph"),
    la.ModularityVertexPartition,
    interslice_weight=1.0,
    n_iterations=-1,
    seed=0,
    vertex_id_attr="_nx_name")

temporal_node_color = {
    t: [c[m] for m in temporal_membership[t]]
    for t in range(len(TG))}

temporal_edge_color = {
    t: get_edge_color(TG[t].edges(), temporal_node_color[t])
    for t in range(len(TG))}

tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    node_size=300,
    temporal_node_color=temporal_node_color,
    temporal_edge_color=temporal_edge_color,
    connectionstyle="arc3,rad=0.1",
    suptitle="Modularity optimization on multislice graph")

Simply considering the network's temporal dimension allows modularity optimization to correctly
retrieve the ground truths in the toy network, while maintaining the community indices fixed over
time.

### Computing modularity and conductance

To better understand the obtained results optimizing static and temporal (multislice) modularity
as quality functions, we may compute the modularity and conductance of the graph with each partitioning:

In [ ]:
static_communities = list(membership)
modularity = tx.modularity(TG, partitions=static_communities)
print(f"Modularity (static partitions): {modularity:.3f}")

temporal_communities = tx.partitions(TG, attr=list(temporal_membership))[-1]
modularity = tx.modularity(TG, partitions=temporal_communities)
print(f"Modularity (temporal partitions): {modularity:.3f}")

We see static modularity yields a higher value considering the network partitioning obtained by
optimizing it on a single graph. The same observation applies to graph conductance [6]:

In [ ]:
conductance = tx.conductance(TG, partitions=static_communities)
print(f"Conductance (static partitions): {conductance:.3f}")

conductance = tx.conductance(TG, partitions=temporal_communities)
print(f"Conductance (temporal partitions): {conductance:.3f}")

Let's now compute the value of temporal (multislice [5] and longitudinal [7]) modularity, considering the
same partitioning schemes:

In [ ]:
ms_modularity = tx.multislice_modularity(TG, partitions=static_communities)
print(f"MS-Modularity (static partitions): {ms_modularity:.3f}")

ms_modularity = tx.multislice_modularity(TG, partitions=temporal_communities)
print(f"MS-Modularity (temporal partitions): {ms_modularity:.3f}")

In [ ]:
l_modularity = tx.modularity(TG.to_static(), partitions=static_communities)
print(f"L-Modularity (static partitions): {l_modularity:.3f}")

l_modularity = tx.modularity(TG, partitions=temporal_communities)
print(f"L-Modularity (temporal partitions): {l_modularity:.3f}")

We see that...

This example showcases how employing time-aware quality functions for the evaluation of community
structure helps in the task of community detection, allowing a better description of the network
for the purposes of exploratory analysis.

___

**References**

1. Freeman, L.C. (1979).
   Centrality in Social Networks I: Conceptual Clarification.
   Social Networks 1, 215-239.

2. Hyoungshick, K., Anderson, R. (2012).
   Temporal node centrality in complex networks.
   Physical Review E, 85(2).

3. V. A. Traag, L. Waltman, N. J. van Eck (2019). ''From Louvain to
   Leiden: guaranteeing well-connected communities''. Scientific Reports,
   9(1), 5233.

4. Tiago P. Peixoto (2023). ''Descriptive Vs. Inferential Community
   Detection in Networks: Pitfalls, Myths and Half-Truths''. Elements in
   the Structure and Dynamics of Complex Networks, Cambridge University
   Press.

5. P. J. Mucha et al (2010). ''Community Structure in Time-Dependent,
   Multiscale, and Multiplex Networks''. Science, 328, 876-878.

6. Kannan, R., Vempala, S., & Vetta, A. (2004). ''On clusterings: Good, bad and spectral''.
   Journal of the ACM (JACM), 51(3), 497-515.

7. V. Brabant et al (2025). ''Longitudinal modularity, a modularity for
   link streams.'' EPJ Data Science, 14, 12.